In [24]:
import tensorflow as tf
from tensorflow.contrib import predictor

export_dir = "model_sr50_new"
predict_fn = predictor.from_saved_model(export_dir)

INFO:tensorflow:Restoring parameters from b'model_sr50_new/variables/variables'


In [25]:
from matplotlib import pyplot as plt
import numpy as np
import random

In [26]:
import tqdm

In [27]:
from time import time

In [28]:
import sys 
sys.path.insert(0,'../..')

In [29]:
from cnf_dataset import clauses_to_matrix
from dpll import DPLL, RandomClauseDPLL, MostCommonVarDPLL, RandomVarDPLL, JeroslowWangDPLL
import cdcl
CDCL = cdcl.CDCL
from cnf import get_random_kcnf, CNF, get_sats_SR, get_pos_SR, get_sat_SR, get_random_sat_kcnf
from collections import Counter

In [30]:
LIMIT_RUNS = 1000

In [31]:
import math
from collections import defaultdict

def jw(clauses):
    score = defaultdict(int)

    for clause in clauses:
        for l in clause:
            score[l] += 2. ** (-len(clause))

    return max(score, key=score.get)

In [32]:
np.set_printoptions(precision=3, suppress=True)

In [33]:
import tensorflow as tf
import os

BATCH_SIZE = 1

In [34]:
VERBOSE = False
TIMEOUT = 10000
cdcl.VERBOSE = VERBOSE
cdcl.TIMEOUT = TIMEOUT

class GraphBasedCDCL(CDCL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        output = predict_fn({"input": inputs})
        policy_probs = output['policy_probabilities']
        sat_prob = output['sat_probabilities'][0]
                
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        if VERBOSE:
            print("Chosen neural", best_svar)
            print("Pred SAT prob", sat_prob)
        return best_svar

class HumbleGraphBasedCDCL(CDCL):
    def suggest(self, input_cnf: CNF):
        clause_num = len(input_cnf.clauses)
        var_num = max(input_cnf.vars)
        inputs = np.asarray([clauses_to_matrix(input_cnf.clauses, clause_num, var_num)] * BATCH_SIZE)
        
        output = predict_fn({"input": inputs})
        policy_probs = output['policy_probabilities']
        sat_prob = output['sat_probabilities'][0]
                
        best_prob = 0.0
        best_svar = None
        for var in input_cnf.vars:
            for svar in [var, -var]:
                svar_prob = policy_probs[0][var-1][0 if svar > 0 else 1]
                if svar_prob > best_prob:
                    best_prob = svar_prob
                    best_svar = svar
        if VERBOSE:
            print("Chosen neural", best_svar)
            print("Pred SAT prob", sat_prob)
        if sat_prob < 0.3:
            best_svar = jw(input_cnf.clauses)
            if VERBOSE:
                print("Choosing JW", best_svar)
        return best_svar

class JeroslawCDCL(CDCL):
    def suggest(self, cnf: CNF):
        res = jw(cnf.clauses)
        if VERBOSE:
            print("Chosen JW", res)
        return res

In [35]:
def compute_steps(sats, cdcl_cls):
    steps = []
    solved = 0
    for sat in tqdm.tqdm(sats):
        cdcl = cdcl_cls()
        res = cdcl.run(sat)
        # assert res is not None
        if res is not None:
            steps.append(cdcl.number_of_runs)
            solved += 1
    print("Within {} steps solved {} problems out of {}".format(LIMIT_RUNS, solved, len(sats)))
    return steps

In [36]:
def compute_and_print_steps(sats, dpll_cls):
    try:
        print("")
        print("Results of {}".format(dpll_cls.__name__))
        steps = compute_steps(sats, dpll_cls)
        print("#Sats: {}; avg step: {:.2f}; stdev step: {:.2f}".format(
            len(steps), np.mean(steps), np.std(steps)))
        print("Table: {}".format(steps))
    except Exception as e:
        print(e)
        print("Timeout!", TIMEOUT, "steps")


In [37]:
# s - number of samples
# n - max number of clauses, use 100 * m
# m - number of variables
def print_all(s, n, m, light=False, seed=1, to_test=[HumbleGraphBasedCDCL, JeroslawCDCL]):
    print("Starting...")
    global S, N, M
    S = s
    N = n # number of clauses
    M = m # number of variables
    
    MAX_TRIES = 100000
    sats = []
    
    random.seed(seed)
    np.random.seed(seed)
    
    for index in range(MAX_TRIES):
        if len(sats) >= S:
            break
        sat = get_pos_SR(M, M, N)
        sats.append(sat)
    assert len(sats) == S
    print("We have generated {} formulas".format(len(sats)))
    for method in to_test:
        compute_and_print_steps(sats, method)


In [38]:
print_all(10, 1000, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Starting...
We have generated 10 formulas

Results of HumbleGraphBasedCDCL


100%|██████████| 10/10 [00:00<00:00, 108.97it/s]

Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 4.60; stdev step: 1.02
Table: [5, 6, 4, 4, 4, 4, 6, 6, 4, 3]

Results of JeroslawCDCL
Within 1000 steps solved 10 problems out of 10
#Sats: 10; avg step: 4.70; stdev step: 0.90
Table: [6, 5, 5, 3, 5, 4, 6, 5, 4, 4]


In [39]:
predict_fn.graph.collections

['global_step',
 'losses',
 'saved_model_main_op',
 'trainable_variables',
 'variables']

In [40]:
predict_fn.fetch_tensors

{'sat_probabilities': <tf.Tensor 'sat_prob_50:0' shape=(?,) dtype=float32>,
 'policy_probabilities': <tf.Tensor 'policy_prob_50:0' shape=(?, ?, 2) dtype=float32>}

In [41]:
print_all(100, 200, 10)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


 22%|██▏       | 22/100 [00:00<00:00, 94.91it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 4.52; stdev step: 1.47
Table: [5, 6, 4, 4, 4, 4, 6, 6, 4, 3, 3, 6, 3, 3, 4, 4, 3, 2, 4, 6, 6, 4, 8, 2, 4, 3, 4, 4, 3, 6, 4, 4, 5, 2, 5, 3, 7, 3, 4, 6, 6, 4, 3, 2, 7, 2, 3, 4, 3, 6, 5, 4, 4, 4, 4, 5, 5, 5, 4, 6, 5, 4, 6, 6, 6, 5, 3, 4, 4, 3, 8, 7, 4, 4, 7, 4, 3, 6, 4, 4, 7, 4, 5, 2, 5, 5, 6, 3, 6, 6, 3, 5, 8, 4, 2, 4, 7, 5, 4, 7]

Results of JeroslawCDCL


100%|██████████| 100/100 [00:01<00:00, 89.23it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 4.98; stdev step: 1.72
Table: [6, 5, 5, 3, 5, 4, 6, 5, 4, 4, 7, 7, 4, 2, 4, 4, 5, 3, 7, 3, 4, 12, 8, 4, 4, 4, 5, 9, 6, 5, 4, 4, 6, 5, 4, 3, 4, 3, 5, 6, 5, 4, 3, 4, 6, 2, 3, 2, 3, 5, 5, 7, 5, 7, 6, 4, 7, 5, 4, 4, 4, 4, 6, 5, 5, 5, 5, 4, 5, 3, 6, 6, 5, 4, 9, 6, 6, 5, 4, 6, 7, 5, 4, 4, 6, 5, 5, 3, 8, 7, 4, 11, 5, 3, 4, 4, 8, 6, 4, 3]


In [42]:
print_all(100, 200, 20)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  2%|▏         | 2/100 [00:00<00:15,  6.14it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 10.29; stdev step: 5.42
Table: [5, 6, 8, 6, 8, 7, 7, 8, 7, 12, 7, 22, 11, 5, 9, 9, 10, 32, 23, 10, 10, 12, 7, 13, 14, 10, 2, 18, 7, 7, 5, 7, 19, 11, 9, 13, 8, 7, 7, 13, 16, 7, 7, 12, 22, 7, 11, 8, 17, 9, 9, 13, 13, 9, 25, 8, 9, 8, 6, 9, 13, 8, 9, 11, 28, 6, 11, 15, 5, 31, 7, 7, 9, 8, 7, 4, 10, 7, 5, 8, 10, 8, 8, 7, 11, 9, 6, 8, 10, 11, 7, 6, 7, 13, 4, 12, 14, 9, 16, 8]

Results of JeroslawCDCL


100%|██████████| 100/100 [00:12<00:00,  8.21it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 12.02; stdev step: 5.74
Table: [5, 14, 8, 7, 13, 8, 7, 12, 18, 23, 12, 15, 23, 4, 15, 6, 15, 14, 8, 10, 9, 7, 7, 11, 14, 5, 4, 14, 4, 8, 8, 15, 6, 8, 7, 13, 8, 9, 8, 7, 17, 6, 12, 18, 14, 22, 8, 13, 11, 14, 13, 13, 18, 15, 15, 14, 14, 12, 19, 20, 12, 11, 17, 11, 25, 19, 5, 17, 14, 33, 7, 11, 4, 6, 7, 8, 5, 11, 7, 9, 15, 22, 7, 8, 23, 9, 6, 11, 16, 12, 7, 7, 25, 10, 15, 27, 10, 8, 20, 8]


In [43]:
print_all(100, 500, 30)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  2%|▏         | 2/100 [00:00<00:29,  3.32it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 15.73; stdev step: 11.88
Table: [9, 23, 25, 15, 14, 26, 13, 12, 8, 14, 10, 11, 13, 15, 18, 9, 36, 11, 7, 15, 37, 21, 14, 15, 11, 13, 9, 12, 12, 15, 14, 12, 10, 7, 15, 57, 9, 12, 8, 10, 12, 8, 32, 14, 11, 12, 11, 15, 70, 10, 31, 12, 10, 66, 16, 8, 9, 12, 10, 13, 15, 13, 12, 49, 9, 15, 10, 8, 9, 18, 10, 14, 11, 10, 32, 17, 9, 10, 10, 14, 10, 10, 12, 14, 9, 10, 8, 14, 16, 6, 14, 10, 7, 9, 55, 4, 23, 8, 27, 13]

Results of JeroslawCDCL


100%|██████████| 100/100 [00:27<00:00,  3.69it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 19.66; stdev step: 13.22
Table: [11, 37, 11, 14, 11, 11, 36, 12, 8, 39, 11, 21, 12, 34, 18, 12, 7, 11, 8, 30, 23, 13, 42, 17, 15, 16, 16, 12, 7, 20, 85, 10, 5, 20, 10, 40, 16, 34, 27, 9, 19, 19, 26, 24, 19, 23, 22, 22, 35, 20, 18, 11, 12, 38, 32, 44, 10, 13, 7, 25, 31, 41, 32, 36, 8, 34, 10, 10, 11, 14, 7, 40, 17, 12, 25, 13, 7, 26, 9, 12, 8, 40, 37, 11, 7, 10, 5, 11, 9, 38, 19, 7, 11, 13, 59, 9, 22, 9, 7, 9]


In [44]:
print_all(100, 10000, 40, light=True)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  0%|          | 0/100 [00:00<?, ?it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 31.09; stdev step: 31.48
Table: [35, 103, 14, 15, 19, 19, 7, 15, 19, 21, 133, 7, 12, 9, 9, 8, 12, 17, 82, 46, 14, 15, 56, 30, 12, 19, 13, 18, 19, 12, 18, 12, 88, 10, 15, 19, 14, 15, 20, 42, 18, 16, 15, 51, 27, 14, 19, 18, 12, 10, 65, 13, 13, 24, 10, 157, 17, 81, 17, 12, 10, 26, 14, 90, 15, 18, 14, 105, 14, 16, 19, 17, 24, 146, 15, 17, 55, 15, 106, 13, 19, 11, 14, 20, 68, 58, 13, 14, 14, 52, 36, 104, 51, 40, 19, 17, 16, 60, 17, 45]

Results of JeroslawCDCL


100%|██████████| 100/100 [01:49<00:00,  1.33s/it]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 41.04; stdev step: 29.05
Table: [51, 74, 58, 11, 11, 36, 50, 45, 25, 42, 18, 52, 54, 9, 16, 15, 20, 25, 68, 51, 21, 42, 18, 27, 48, 17, 29, 12, 45, 11, 33, 27, 56, 27, 18, 34, 13, 68, 68, 14, 21, 55, 18, 20, 28, 60, 42, 12, 29, 9, 13, 15, 9, 20, 43, 124, 46, 103, 14, 10, 12, 34, 111, 107, 13, 30, 21, 63, 45, 51, 74, 85, 27, 30, 20, 103, 66, 17, 52, 27, 64, 15, 26, 13, 84, 28, 14, 63, 18, 149, 45, 68, 73, 102, 16, 41, 69, 89, 27, 62]


In [45]:
print_all(100, 10000, 50, light=True)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  0%|          | 0/100 [00:00<?, ?it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 47.08; stdev step: 58.60
Table: [17, 8, 21, 26, 25, 18, 32, 20, 15, 20, 24, 23, 56, 15, 186, 19, 87, 31, 21, 220, 21, 17, 13, 15, 15, 22, 71, 15, 24, 27, 18, 14, 16, 9, 18, 186, 25, 264, 15, 17, 44, 59, 18, 111, 23, 17, 16, 110, 171, 79, 12, 30, 13, 98, 18, 18, 30, 74, 24, 34, 20, 36, 11, 30, 15, 12, 31, 15, 23, 21, 100, 21, 275, 19, 25, 62, 17, 180, 12, 10, 240, 114, 18, 23, 14, 22, 32, 171, 17, 17, 18, 19, 57, 27, 46, 15, 144, 13, 12, 49]

Results of JeroslawCDCL


100%|██████████| 100/100 [04:14<00:00,  2.42s/it]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 71.66; stdev step: 58.37
Table: [29, 12, 91, 46, 56, 125, 56, 14, 107, 17, 26, 76, 172, 116, 288, 109, 87, 189, 19, 58, 40, 25, 21, 13, 25, 33, 97, 39, 229, 36, 40, 22, 11, 40, 20, 182, 108, 38, 40, 21, 92, 84, 79, 66, 31, 216, 62, 133, 262, 77, 14, 30, 8, 84, 19, 48, 74, 24, 20, 13, 108, 21, 26, 45, 67, 11, 34, 16, 111, 26, 19, 36, 17, 74, 132, 50, 102, 159, 144, 12, 136, 125, 41, 112, 124, 34, 75, 99, 102, 18, 61, 83, 151, 164, 38, 94, 141, 16, 54, 79]


In [46]:
print_all(100, 10000, 70, light=True)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  0%|          | 0/100 [00:00<?, ?it/s]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 214.66; stdev step: 377.26
Table: [29, 138, 16, 18, 41, 23, 30, 24, 30, 2864, 220, 607, 24, 30, 425, 27, 26, 68, 94, 322, 32, 36, 48, 143, 21, 431, 102, 17, 308, 34, 825, 19, 28, 457, 23, 129, 30, 1228, 215, 21, 283, 66, 1241, 33, 29, 36, 26, 491, 38, 32, 63, 825, 27, 254, 59, 1050, 69, 34, 497, 69, 730, 251, 29, 18, 21, 102, 29, 308, 552, 339, 44, 53, 18, 40, 43, 279, 19, 124, 31, 25, 34, 236, 29, 45, 25, 29, 423, 379, 520, 43, 31, 422, 26, 70, 235, 24, 473, 31, 883, 28]

Results of JeroslawCDCL


100%|██████████| 100/100 [27:45<00:00, 16.78s/it] 

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 280.48; stdev step: 270.21
Table: [361, 244, 63, 159, 35, 141, 41, 30, 360, 882, 69, 256, 577, 649, 451, 38, 367, 78, 450, 87, 185, 84, 154, 184, 58, 82, 77, 20, 62, 170, 1291, 45, 327, 368, 401, 207, 269, 1073, 158, 327, 720, 1193, 877, 91, 193, 380, 16, 236, 322, 388, 20, 403, 179, 602, 55, 764, 271, 156, 428, 303, 37, 386, 275, 80, 14, 112, 134, 545, 83, 42, 1030, 55, 56, 471, 107, 197, 145, 142, 26, 101, 68, 274, 173, 228, 178, 284, 452, 202, 277, 71, 676, 423, 181, 89, 389, 128, 377, 168, 845, 50]


In [47]:
print_all(100, 10000, 90, light=True)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


  1%|          | 1/100 [02:13<3:40:57, 133.91s/it]

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 673.82; stdev step: 988.60
Table: [30, 315, 29, 32, 34, 665, 39, 36, 35, 30, 464, 1197, 1053, 51, 106, 43, 225, 513, 908, 28, 1471, 898, 328, 635, 41, 684, 966, 5570, 1756, 29, 927, 2697, 2635, 51, 716, 40, 37, 1241, 33, 486, 1305, 175, 42, 28, 41, 39, 35, 1746, 186, 54, 743, 1350, 52, 174, 125, 406, 659, 39, 3248, 58, 355, 1069, 1568, 36, 3488, 41, 1018, 25, 43, 36, 913, 4201, 427, 999, 341, 267, 42, 42, 39, 29, 489, 2116, 54, 89, 790, 169, 327, 40, 2109, 2080, 41, 258, 2875, 333, 888, 1893, 38, 28, 177, 30]

Results of JeroslawCDCL


100%|██████████| 100/100 [2:22:05<00:00, 37.33s/it]  

Within 1000 steps solved 100 problems out of 100
#Sats: 100; avg step: 897.07; stdev step: 798.36
Table: [1561, 971, 474, 44, 3957, 1352, 119, 978, 31, 114, 1006, 489, 1075, 117, 889, 30, 482, 1242, 2435, 131, 1106, 696, 29, 1973, 424, 1485, 347, 903, 1231, 436, 501, 1691, 2174, 31, 2151, 899, 63, 1115, 1150, 1934, 1623, 1019, 1853, 243, 207, 22, 794, 2600, 56, 419, 123, 1264, 552, 705, 705, 273, 1783, 344, 1770, 208, 883, 726, 92, 228, 3253, 52, 369, 403, 421, 286, 29, 2135, 2029, 893, 2088, 106, 510, 1325, 178, 1066, 1527, 434, 862, 378, 313, 414, 550, 1271, 2443, 1273, 1764, 1332, 1540, 1901, 112, 1763, 162, 27, 67, 103]


In [ ]:
print_all(100, 10000, 110, light=True)

Starting...


  0%|          | 0/100 [00:00<?, ?it/s]

We have generated 100 formulas

Results of HumbleGraphBasedCDCL


 14%|█▍        | 14/100 [11:45:42<62:06:04, 2599.59s/it] 